In [1]:
import os
from dotenv import load_dotenv
import requests
from datetime import datetime
from sys import version_info
from hashlib import md5


load_dotenv()

API_KEY = os.getenv('API_KEY')
DEV_ID = os.getenv('DEV_ID')
base_url = "https://api.smitegame.com/smiteapi.svc"
platform = "pc"
language = "en"

In [2]:
class HiRezAPI:
    def __init__ (self, dev_id, auth_key, *args, **kw):
        self.auth_key = auth_key;
        self.dev_id = dev_id;
        self.endpoint = kw.pop('endpoint', 'https://api.smitegame.com/smiteapi.svc') 
        self.headers = kw.pop('headers', {})
        if 'user-agent' not in self.headers:
            self.headers['user-agent'] = f'HiRezAPIWrapper [Python/{version_info.major}.{version_info.minor}]'

    def request(self, url, *args, **kw):
        r = requests.get(url=url, headers={**self.headers, **kw.pop('headers', {})}, *args, **kw)
        if r.headers.get('Content-Type', '').rfind('json') != -1:
            return r.json()
        return r.text

    def generate_signature(self, method, timestamp=None, *, encoding=None):
        return md5(f'{self.dev_id}{method.lower()}{self.auth_key}{timestamp or self.timestamp()}'.encode(encoding or 'utf-8')).hexdigest()

    def timestamp(self, _format=None):
        return datetime.utcnow().strftime(_format or '%Y%m%d%H%M%S')

    def create_session(self):
        r = self.request(f'{self.endpoint}/createsessionjson/{self.dev_id}/{self.generate_signature("createsession")}/{self.timestamp()}')
        if r.get('ret_msg', '').lower() == 'approved':
            self.session_id = r.get('session_id')

    def ping(self):
        return self.request(f'{self.endpoint}/pingjson')

    def test_session(self, session_id=None):
        return self.request(f'{self.endpoint}/testsessionjson/{self.dev_id}/{self.generate_signature("testsession")}/{session_id or self.session_id}/{self.timestamp()}')

    def data_used(self):
        return self.request(f'{self.endpoint}/getdatausedjson/{self.dev_id}/{self.generate_signature("getdataused")}/{self.session_id}/{self.timestamp()}')

    def server_status(self):
        return self.request(f'{self.endpoint}/gethirezserverstatusjson/{self.dev_id}/{self.generate_signature("gethirezserverstatus")}/{self.session_id}/{self.timestamp()}')

    def patch_info(self):
        return self.request (f'{self.endpoint}/getpatchinfojson/{self.dev_id}/{self.generate_signature("getpatchinfo")}/{self.session_id}/{self.timestamp()}')

    def get_gods(self):
        return self.request (f'{self.endpoint}/getgodsjson/{self.dev_id}/{self.generate_signature("getgods")}/{self.session_id}/{self.timestamp()}/1')

    def get_playeridbyname(self, name):
        return self.request (f'{self.endpoint}/getplayeridbynamejson/{self.dev_id}/{self.generate_signature("getplayeridbyname")}/{self.session_id}/{self.timestamp()}/{name}')
    
    def get_playerachievements(self, playerid):
        return self.request (f'{self.endpoint}/getplayerachievementsjson/{self.dev_id}/{self.generate_signature("getplayerachievements")}/{self.session_id}/{self.timestamp()}/{playerid}')
    
hirez_api = HiRezAPI(dev_id=4529, auth_key='F43736C8C21741D89BEACE9705CD437F')
hirez_api.create_session()
"""
print(hirez_api.data_used())
print(hirez_api.server_status())
print(hirez_api.patch_info())
print(hirez_api.ping())
print(hirez_api.test_session())
"""

'\nprint(hirez_api.data_used())\nprint(hirez_api.server_status())\nprint(hirez_api.patch_info())\nprint(hirez_api.ping())\nprint(hirez_api.test_session())\n'

In [3]:
hirez_api.get_gods()

In [4]:
player = hirez_api.get_playeridbyname("zapman")

In [5]:
hirez_api.get_playerachievements(player[0]['player_id'])

{'AssistedKills': 148525,
 'CampsCleared': 26296,
 'Deaths': 124007,
 'DivineSpree': 532,
 'DoubleKills': 10738,
 'FireGiantKills': 2999,
 'FirstBloods': 3707,
 'GodLikeSpree': 61,
 'GoldFuryKills': 6704,
 'Id': 367737,
 'ImmortalSpree': 166,
 'KillingSpree': 11990,
 'MinionKills': 3597610,
 'Name': 'Zapman',
 'PentaKills': 18,
 'PhoenixKills': 4926,
 'PlayerKills': 118230,
 'QuadraKills': 165,
 'RampageSpree': 3951,
 'ShutdownSpree': 2286,
 'SiegeJuggernautKills': 12,
 'TowerKills': 24693,
 'TripleKills': 1300,
 'UnstoppableSpree': 1442,
 'WildJuggernautKills': 7,
 'ret_msg': None}